In [1]:
import requests
import json
import pandas as pd
import numpy as np
import time
import helper_functions as hf
from helper_functions import *

In [2]:
from importlib import reload
reload(helper_functions)
from helper_functions import *

NameError: name 'helper_functions' is not defined

Help on function get_list in module helper_functions:

get_list(book_list_name, api_key)
    Function to compose and obtain information from a NYT
    books endpoint.
    
    There are many book lists, and the api returns the book lists
    in JSON format with a lot of information beyond just the titles 
    of the books. 
    
    This function returns a book list with just the pertinent 
    information. a dictionary in the format {ISBN:Book title}. 
    
    Dependencies: 
    requests
    json



In [44]:
dir(hf)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'aggregate',
 'dictlist_to_df',
 'get_list',
 'test']

## Obtain NYT list information
I'm trying to get bestseller information from the nyt API. They have a variety of lists, and metadata on each list that they have. The metadata is stored in the '/lists/names.json' endpoint. Lets view this metadata first (it will help with looping functionality later when we need details from each sub-list) 

In [45]:
from secrets import NYT_API_KEY

In [46]:
book_lists = requests.get('https://api.nytimes.com/svc/books/v3/lists/names.json?api-key='+NYT_API_KEY)

In [47]:
listoflists = json.loads(book_lists.text)['results']

In [48]:
listoflists[0]

{'list_name': 'Combined Print and E-Book Fiction',
 'display_name': 'Combined Print & E-Book Fiction',
 'list_name_encoded': 'combined-print-and-e-book-fiction',
 'oldest_published_date': '2011-02-13',
 'newest_published_date': '2020-03-08',
 'updated': 'WEEKLY'}

In [49]:
aggregate(listoflists,'list_name')[:5]

['Combined Print and E-Book Fiction',
 'Combined Print and E-Book Nonfiction',
 'Hardcover Fiction',
 'Hardcover Nonfiction',
 'Trade Fiction Paperback']

In [50]:
# alternativley we can just read the dictionary:
lists_df = pd.DataFrame(listoflists)
lists = lists_df['list_name_encoded'].to_list()

In [51]:
lists[:5]

['combined-print-and-e-book-fiction',
 'combined-print-and-e-book-nonfiction',
 'hardcover-fiction',
 'hardcover-nonfiction',
 'trade-fiction-paperback']

## Get information from NYT API
Here we retreive information from a specific list. the current hardcover bestsellers.

In [4]:
a = requests.get('https://api.nytimes.com/svc/books/v3/lists/current/hardcover-fiction.json?api-key='+NYT_API_KEY)

In [57]:
a.text

'{"status":"OK","copyright":"Copyright (c) 2020 The New York Times Company.  All Rights Reserved.","num_results":15,"last_modified":"2020-02-26T23:38:02-05:00","results":{"list_name":"Hardcover Fiction","list_name_encoded":"hardcover-fiction","bestsellers_date":"2020-02-22","published_date":"2020-03-08","published_date_description":"latest","next_published_date":"","previous_published_date":"2020-03-01","display_name":"Hardcover Fiction","normal_list_ends_at":15,"updated":"WEEKLY","books":[{"rank":1,"rank_last_week":1,"weeks_on_list":77,"asterisk":0,"dagger":0,"primary_isbn10":"0735219095","primary_isbn13":"9780735219090","publisher":"Putnam","description":"In a quiet town on the North Carolina coast in 1969, a young woman who survived alone in the marsh becomes a murder suspect.","price":0,"title":"WHERE THE CRAWDADS SING","author":"Delia Owens","contributor":"by Delia Owens","contributor_note":"","book_image":"https:\\/\\/s1.nyt.com\\/du\\/books\\/images\\/9780735219090.jpg","book_im

In [56]:
b

{'status': 'OK',
 'copyright': 'Copyright (c) 2020 The New York Times Company.  All Rights Reserved.',
 'num_results': 15,
 'last_modified': '2020-02-26T23:38:02-05:00',
 'results': {'list_name': 'Hardcover Fiction',
  'list_name_encoded': 'hardcover-fiction',
  'bestsellers_date': '2020-02-22',
  'published_date': '2020-03-08',
  'published_date_description': 'latest',
  'next_published_date': '',
  'previous_published_date': '2020-03-01',
  'display_name': 'Hardcover Fiction',
  'normal_list_ends_at': 15,
  'updated': 'WEEKLY',
  'books': [{'rank': 1,
    'rank_last_week': 1,
    'weeks_on_list': 77,
    'asterisk': 0,
    'dagger': 0,
    'primary_isbn10': '0735219095',
    'primary_isbn13': '9780735219090',
    'publisher': 'Putnam',
    'description': 'In a quiet town on the North Carolina coast in 1969, a young woman who survived alone in the marsh becomes a murder suspect.',
    'price': 0,
    'title': 'WHERE THE CRAWDADS SING',
    'author': 'Delia Owens',
    'contributor': '

In [10]:
b.keys()

dict_keys(['status', 'copyright', 'num_results', 'last_modified', 'results'])

In [26]:
b['results'].keys()

dict_keys(['list_name', 'list_name_encoded', 'bestsellers_date', 'published_date', 'published_date_description', 'next_published_date', 'previous_published_date', 'display_name', 'normal_list_ends_at', 'updated', 'books', 'corrections'])

In [21]:
c = b['results']

In [40]:
c['books'][0].keys()

dict_keys(['rank', 'rank_last_week', 'weeks_on_list', 'asterisk', 'dagger', 'primary_isbn10', 'primary_isbn13', 'publisher', 'description', 'price', 'title', 'author', 'contributor', 'contributor_note', 'book_image', 'book_image_width', 'book_image_height', 'amazon_product_url', 'age_group', 'book_review_link', 'first_chapter_link', 'sunday_review_link', 'article_chapter_link', 'isbns', 'buy_links', 'book_uri'])

In [22]:
# One data entry of a book. 
b['results']['books'][0]

{'rank': 1,
 'rank_last_week': 4,
 'weeks_on_list': 76,
 'asterisk': 0,
 'dagger': 0,
 'primary_isbn10': '0735219095',
 'primary_isbn13': '9780735219090',
 'publisher': 'Putnam',
 'description': 'In a quiet town on the North Carolina coast in 1969, a young woman who survived alone in the marsh becomes a murder suspect.',
 'price': 0,
 'title': 'WHERE THE CRAWDADS SING',
 'author': 'Delia Owens',
 'contributor': 'by Delia Owens',
 'contributor_note': '',
 'book_image': 'https://s1.nyt.com/du/books/images/9780735219090.jpg',
 'book_image_width': 328,
 'book_image_height': 495,
 'amazon_product_url': 'https://www.amazon.com/Where-Crawdads-Sing-Delia-Owens/dp/0735219095?tag=NYTBS-20',
 'age_group': '',
 'book_review_link': '',
 'first_chapter_link': '',
 'sunday_review_link': '',
 'article_chapter_link': '',
 'isbns': [{'isbn10': '0735219095', 'isbn13': '9780735219090'},
  {'isbn10': '0735219117', 'isbn13': '9780735219113'},
  {'isbn10': '0525640371', 'isbn13': '9780525640370'},
  {'isbn10

In [43]:
books = []
for i in b['results']['books']:
    books.append(i['title'])

In [44]:
books

['WHERE THE CRAWDADS SING',
 'AMERICAN DIRT',
 'THE SILENT PATIENT',
 'ONE MINUTE OUT',
 'GOLDEN IN DEATH',
 'THE DUTCH HOUSE',
 'SUCH A FUN AGE',
 'A LONG PETAL OF THE SEA',
 'THE GUARDIANS',
 'THE GIVER OF STARS',
 'THE SUN DOWN MOTEL',
 'CROOKED RIVER',
 'THE OTHER MRS.',
 'THE HOLDOUT',
 'DEAR EDWARD']

##  Getting all lists from a time period

In [30]:
lists_df.head()

,display_name,list_name,list_name_encoded,newest_published_date,oldest_published_date,updated
0,Combined Print & E-Book Fiction,Combined Print and E-Book Fiction,combined-print-and-e-book-fiction,2020-03-01,2011-02-13,WEEKLY
1,Combined Print & E-Book Nonfiction,Combined Print and E-Book Nonfiction,combined-print-and-e-book-nonfiction,2020-03-01,2011-02-13,WEEKLY
2,Hardcover Fiction,Hardcover Fiction,hardcover-fiction,2020-03-01,2008-06-08,WEEKLY
3,Hardcover Nonfiction,Hardcover Nonfiction,hardcover-nonfiction,2020-03-01,2008-06-08,WEEKLY
4,Paperback Trade Fiction,Trade Fiction Paperback,trade-fiction-paperback,2020-03-01,2008-06-08,WEEKLY


In [31]:
lists_df['book_list']=np.nan

In [32]:
lists_df.head()

,display_name,list_name,list_name_encoded,newest_published_date,oldest_published_date,updated,book_list
0,Combined Print & E-Book Fiction,Combined Print and E-Book Fiction,combined-print-and-e-book-fiction,2020-03-01,2011-02-13,WEEKLY,NaN
1,Combined Print & E-Book Nonfiction,Combined Print and E-Book Nonfiction,combined-print-and-e-book-nonfiction,2020-03-01,2011-02-13,WEEKLY,NaN
2,Hardcover Fiction,Hardcover Fiction,hardcover-fiction,2020-03-01,2008-06-08,WEEKLY,NaN
3,Hardcover Nonfiction,Hardcover Nonfiction,hardcover-nonfiction,2020-03-01,2008-06-08,WEEKLY,NaN
4,Paperback Trade Fiction,Trade Fiction Paperback,trade-fiction-paperback,2020-03-01,2008-06-08,WEEKLY,NaN


In [33]:
# This doesn't work, I have to figure out why.
lists_df.loc[lists_df.list_name_encoded=='hardcover-fiction','book_list'] = ['test',"this"]

ValueError: Must have equal len keys and value when setting with an iterable

In [34]:
lists_df.head()

,display_name,list_name,list_name_encoded,newest_published_date,oldest_published_date,updated,book_list
0,Combined Print & E-Book Fiction,Combined Print and E-Book Fiction,combined-print-and-e-book-fiction,2020-03-01,2011-02-13,WEEKLY,NaN
1,Combined Print & E-Book Nonfiction,Combined Print and E-Book Nonfiction,combined-print-and-e-book-nonfiction,2020-03-01,2011-02-13,WEEKLY,NaN
2,Hardcover Fiction,Hardcover Fiction,hardcover-fiction,2020-03-01,2008-06-08,WEEKLY,NaN
3,Hardcover Nonfiction,Hardcover Nonfiction,hardcover-nonfiction,2020-03-01,2008-06-08,WEEKLY,NaN
4,Paperback Trade Fiction,Trade Fiction Paperback,trade-fiction-paperback,2020-03-01,2008-06-08,WEEKLY,NaN


In [35]:
lists1 = lists[:2]
lists1

['combined-print-and-e-book-fiction', 'combined-print-and-e-book-nonfiction']

In [52]:
# Retrive information 
books_dict2 = {}
for item in lists:
    request_url="https://api.nytimes.com/svc/books/v3/lists/current/"+item+".json?api-key="+NYT_API_KEY
    results = requests.get(request_url)
    results = json.loads(results.text)
    print("status:", results['status'])
    try:
        book_list = aggregate(results['results']['books'],'title')
        books_dict2[item]=book_list
        print(item, "added")
    except KeyError:
        print(item, "NOT added")
    time.sleep(6)

status: OK
combined-print-and-e-book-fiction added
status: OK
combined-print-and-e-book-nonfiction added
status: OK
hardcover-fiction added
status: OK
hardcover-nonfiction added
status: OK
trade-fiction-paperback added
status: OK
mass-market-paperback added
status: OK
paperback-nonfiction added
status: OK
e-book-fiction added
status: OK
e-book-nonfiction added
status: OK
hardcover-advice added
status: OK
paperback-advice added
status: OK
advice-how-to-and-miscellaneous added
status: OK
hardcover-graphic-books added
status: OK
paperback-graphic-books added
status: OK
manga added
status: OK
combined-print-fiction added
status: OK
combined-print-nonfiction added
status: OK
chapter-books added
status: OK
childrens-middle-grade added
status: OK
childrens-middle-grade-e-book added
status: OK
childrens-middle-grade-hardcover added
status: OK
childrens-middle-grade-paperback added
status: OK
paperback-books added
status: OK
picture-books added
status: OK
series-books added
status: OK
young-adu

In [46]:
lists[:10]

['combined-print-and-e-book-fiction',
 'combined-print-and-e-book-nonfiction',
 'hardcover-fiction',
 'hardcover-nonfiction',
 'trade-fiction-paperback',
 'mass-market-paperback',
 'paperback-nonfiction',
 'e-book-fiction',
 'e-book-nonfiction',
 'hardcover-advice']

In [54]:
books_dict2

{'combined-print-and-e-book-fiction': ['ONE MINUTE OUT',
  'AMERICAN DIRT',
  'WHERE THE CRAWDADS SING',
  'LITTLE FIRES EVERYWHERE',
  'CHASING CASSANDRA',
  'THE SILENT PATIENT',
  'GOLDEN IN DEATH',
  'THE DUTCH HOUSE',
  'THE GIVER OF STARS',
  'THE TATTOOIST OF AUSCHWITZ',
  'THE GUARDIANS',
  'THE OTHER MRS.',
  'THE OUTSIDER',
  'SUCH A FUN AGE',
  'A LONG PETAL OF THE SEA'],
 'combined-print-and-e-book-nonfiction': ['THE MAMBA MENTALITY',
  'DARK TOWERS',
  'OPEN BOOK',
  'A VERY STABLE GENIUS',
  'UN-TRUMPING AMERICA',
  'UNTIL THE END OF TIME',
  'EDUCATED',
  'PROFILES IN CORRUPTION',
  'ON TYRANNY',
  'JUST MERCY',
  'TALKING TO STRANGERS',
  'SAPIENS',
  'BECOMING',
  'THE BODY KEEPS THE SCORE',
  'YOU NEVER FORGET YOUR FIRST'],
 'hardcover-fiction': ['WHERE THE CRAWDADS SING',
  'AMERICAN DIRT',
  'THE SILENT PATIENT',
  'ONE MINUTE OUT',
  'GOLDEN IN DEATH',
  'THE DUTCH HOUSE',
  'SUCH A FUN AGE',
  'A LONG PETAL OF THE SEA',
  'THE GUARDIANS',
  'THE GIVER OF STARS',
 

In [ ]:

request_url="https://api.nytimes.com/svc/books/v3/lists/current/hardcover-political-books.json?api-key=76quqESu9qYqfKpEfnJou4kpY1uJ9pMV"
results = requests.get(request_url)
results = json.loads(results.text)

In [ ]:
aggregate(results['results']['books'],'title')

In [ ]:
results['status']

In [ ]:
df = pd.DataFrame(columns=['test'])
df['test']=lists

In [ ]:
df

In [ ]:
Wikipedia API Information: https://www.mediawiki.org/wiki/API:Main_page
Wikipedia page of interest: https://en.wikipedia.org/wiki/List_of_best-selling_books

Basic Features:

- Title
- Author
    - Gender
    - Nationality
    - Languages Spoken By author (at time of writing)
- Genre
- Publisher
- Published year
- Published in 
    - (country, state city?)
- Pages
- Sales Data
- Original Language

Possible Engineered Features:

- popularity
- Combined Rating Weighed from different sources
- NLP
    - publisher site information
    - book author information
    - Reviews (from one or several sources)
    - Book wiki page
    - Author wiki page
    - Reddit information
    - books reddit search results + sentiment analysis on results
- does reddit exist
- does wiki exist
- does fandom exist
- twitter mentions
- google mentions 
- NN on book cover


In [64]:
returns = requests.get("https://api.nytimes.com/svc/books/v3/lists/2019-01-21/hardcover-fiction.json?api-key="
                 + NYT_API_KEY)

In [65]:
returns.text

'{"status":"OK","copyright":"Copyright (c) 2020 The New York Times Company.  All Rights Reserved.","num_results":15,"last_modified":"2019-08-29T20:54:03-04:00","results":{"list_name":"Hardcover Fiction","list_name_encoded":"hardcover-fiction","bestsellers_date":"2019-01-12","published_date":"2019-01-27","published_date_description":"","next_published_date":"2019-02-03","previous_published_date":"2019-01-20","display_name":"Hardcover Fiction","normal_list_ends_at":15,"updated":"WEEKLY","books":[{"rank":1,"rank_last_week":0,"weeks_on_list":1,"asterisk":0,"dagger":0,"primary_isbn10":"1250133734","primary_isbn13":"9781250133731","publisher":"St. Martin\'s","description":"Jessica Farris\\u2019s life unravels when she signs up for Dr. Shields\\u2019s psychology study.","price":0,"title":"AN ANONYMOUS GIRL","author":"Greer Hendricks and Sarah Pekkanen","contributor":"by Greer Hendricks and Sarah Pekkanen","contributor_note":"","book_image":"https:\\/\\/s1.nyt.com\\/du\\/books\\/images\\/97812

## Rate limitations 

NYT imposes a rate limitation of 10 calls per minute ( 6 second delay for continuous calling ) and 4000 calls a day. This is a reasonable time span, and the rate limits need to be accounted for once the program is ready to index all the data from NYT. 